# Fouille de texte avec NLTK

## Extraction de texte

### ... depuis le web

In [4]:
from urllib.request import urlopen

In [5]:
url = "http://news.bbc.co.uk/2/hi/health/2284783.stm"
html = urlopen(url).read()

In [6]:
# Décommenter la ligne en dessous pour l'affichage
#print(html)

On nettoie les caractères HTML grâce à BeautifulSoup qui reprend une implémentation précédente de NLTK

In [7]:
from bs4 import BeautifulSoup

In [8]:
soup = BeautifulSoup(html)
text2 = soup.get_text()

In [9]:
# Décommenter la ligne en dessous pour l'affichage
#print(text2)

Mais il reste beaucoup de caractère spéciaux tels que les retours charriot \n ou \r ...

In [8]:
text2 = text2.replace('\n', ' ').replace('\r', '')

In [10]:
# Décommenter la ligne en dessous pour l'affichage
#print(text2)

###  ... depuis un fichier stocké sur disque

In [11]:
import lxml
import zipfile

#### Cas d'un fichier .docx

On utilise la lib python-docx qui n'est pas présent par défaut sur Anaconda... Ii faut donc l'installer via pip. On se met en mode magic cell pour exécuter des commandes linux depuis un Jupyter Notebook

In [12]:
%%bash
pip install python-docx

In [13]:
from docx import Document

In [14]:
doc = Document("Virtual Sprint Big Data paper.docx")

In [15]:
texts = []
for para in doc.paragraphs:
    texts.append(para.text)
docxtext = '\n'.join(texts)

In [16]:
docxtext = docxtext.replace('\n', ' ').replace('\r', '')

In [17]:
# Décommenter la ligne en dessous pour l'affichage
#print(docxtext)

## Tokenisation

La fonction nltk.word_tokenize() permet de tokeniser un texte en se basant sur les espaces et les signes de ponctuation. Tokenisons le texte extrait du .docx précédent ...

In [19]:
from nltk import word_tokenize

In [20]:
docx_word_tokens = word_tokenize(docxtext)

In [21]:
# Décommenter la ligne en dessous pour l'affichage
#print(docx_word_tokens)

L'objet TweetTokenizer est spécialement fait pour tokeniser des tweets (reconnaissance des hashtags et des smileys...)

In [22]:
from nltk.tokenize import TweetTokenizer

In [23]:
tweet_tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)

In [24]:
tweet = "This is a cooool #dummysmiley: :-) :-P <3 and some arrows < > -> <--"

In [25]:
tweet_tokens = tweet_tokenizer.tokenize(tweet)

In [26]:
# Décommenter la ligne en dessous pour l'affichage
#print(tweet_tokens)

L'objet MWETokenizer permet de "retokeniser" ou "redistribuer des tokens" en regroupant les expressions composées comme par exemple "a little bit", "in spite of" ...

In [27]:
from nltk.tokenize import MWETokenizer

In [28]:
mwe_tokenizer = MWETokenizer([('a', 'little'), ('a', 'little', 'bit'), ('a', 'lot')])
mwe_tokenizer.add_mwe(('in', 'spite', 'of'))

In [30]:
mwe_test = "In a little or a little bit or a lot in spite of" 

Voilà ce que ça donne avec un tokenizer classique...

In [31]:
word_tokens = word_tokenize(mwe_test)

In [32]:
print(word_tokens)

['In', 'a', 'little', 'or', 'a', 'little', 'bit', 'or', 'a', 'lot', 'in', 'spite', 'of']


Voilà ce que ça donne avec un MWE tokenizer ...

In [33]:
mwe_tokens = mwe_tokenizer.tokenize(word_tokens)

In [35]:
print(mwe_tokens)

['In', 'a_little', 'or', 'a_little_bit', 'or', 'a_lot', 'in_spite_of']


L'objet RegexpTokenizer permet de tokeniser en se basant sur des expressions régulières

In [36]:
from nltk.tokenize import RegexpTokenizer

... soit les tokens correspondent à une expression régulière

In [37]:
s = "Good muffins cost $3.88\nin New York.  Please buy me\ntwo of them.\n\nThanks."

In [38]:
regex_tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
regex_tokenizer.tokenize(s)

['Good',
 'muffins',
 'cost',
 '$3.88',
 'in',
 'New',
 'York',
 '.',
 'Please',
 'buy',
 'me',
 'two',
 'of',
 'them',
 '.',
 'Thanks',
 '.']

... soit le(s) séparateur(s) correspondent à une expression régulière

In [39]:
regex_tokenizer = RegexpTokenizer('\.|\n', gaps=True)
regex_tokenizer.tokenize(s)

['Good muffins cost $3',
 '88',
 'in New York',
 '  Please buy me',
 'two of them',
 'Thanks']

L'objet Blankline Tokenizer permet de tokeniser ligne par ligne

In [40]:
from nltk.tokenize import BlanklineTokenizer

In [41]:
BlanklineTokenizer().tokenize(s)

['Good muffins cost $3.88\nin New York.  Please buy me\ntwo of them.',
 'Thanks.']

## Opérations sur les Tokens

### e.g. filtrage de stopwords et signes de ponctuation

In [42]:
from nltk.corpus import stopwords
import string
from nltk import FreqDist

Ci-dessous, on calcule la fréquence de chaque token (mot) dans le texte docxtext, elle sont affichées dans l'ordre décroissant du plus fréquent au moins fréquent.

In [44]:
FreqDist(docxtext)

FreqDist({' ': 7921, 'e': 4518, 't': 4316, 'a': 3819, 'i': 3542, 's': 3150, 'o': 2960, 'n': 2697, 'r': 2327, 'l': 1548, ...})

Ci-dessous, on filtre les tokens pour ne garder que les mots importants et supprimer les Stopwords (articles, prépositions etc) ainsi que les signes de ponctuation. <br/>
Attention à télécharger préalablement les corpus <b> stopwords </b> et <b> punkt </b> depuis <b> nltk.download() </b>

In [45]:
clean_docxtext_tokens = [item for item in docx_word_tokens
                         if item not in stopwords.words('english') and item not in  string.punctuation]

Recalculons les fréquences des tokens restants, et remarquons qu'on obtient en haut de la chaîne des mots qui sont beaucoup plus pertinents et caractéristique du sujet du document (Big Data)

In [46]:
FreqDist(clean_docxtext_tokens)

FreqDist({'Data': 166, 'Big': 152, 'data': 124, 'statistical': 123, 'organizations': 93, 'The': 64, 'sources': 47, 'use': 41, 'may': 40, 'information': 34, ...})

## Le Part-Of-Speech (POS) Tagging

### Exploration des Tagged Corpora de NLTK

POS Tagging utilisant la fonction pos_tag(). Celle-ci fait appel à l'algorithme du Perceptron pour la reconnaissance des POS Tags.

In [49]:
tokens = word_tokenize("And now for something completely different")

In [50]:
print(tokens)

['And', 'now', 'for', 'something', 'completely', 'different']


In [51]:
from nltk import pos_tag

In [52]:
pos_tag(tokens)

[('And', 'CC'),
 ('now', 'RB'),
 ('for', 'IN'),
 ('something', 'NN'),
 ('completely', 'RB'),
 ('different', 'JJ')]

In [53]:
tokens = word_tokenize("They refuse to permit us to obtain the refuse permit")

L'argument <em> tagset='universal' </em> permet de renvoyer des tags dans un format plus lisible (plus Human Friendly) <br>
Ne pas oublier de télécharger le corpus qui contient ces tags à savoir <b> universal_tagset </b>

In [61]:
#import nltk
#nltk.download('universal_tagset')

In [62]:
pos_tag(tokens,tagset='universal')

[('They', 'PRON'),
 ('refuse', 'VERB'),
 ('to', 'PRT'),
 ('permit', 'VERB'),
 ('us', 'PRON'),
 ('to', 'PRT'),
 ('obtain', 'VERB'),
 ('the', 'DET'),
 ('refuse', 'NOUN'),
 ('permit', 'NOUN')]

Explorons maintenant qq Tagged Corpora de NLTK ...

In [63]:
from nltk.corpus import brown, nps_chat, treebank

In [64]:
brown.tagged_words(tagset='universal')[:10]

[('The', 'DET'),
 ('Fulton', 'NOUN'),
 ('County', 'NOUN'),
 ('Grand', 'ADJ'),
 ('Jury', 'NOUN'),
 ('said', 'VERB'),
 ('Friday', 'NOUN'),
 ('an', 'DET'),
 ('investigation', 'NOUN'),
 ('of', 'ADP')]

In [65]:
nps_chat.tagged_words(tagset='universal')[:10]

[('now', 'ADV'),
 ('im', 'PRON'),
 ('left', 'VERB'),
 ('with', 'ADP'),
 ('this', 'DET'),
 ('gay', 'ADJ'),
 ('name', 'NOUN'),
 (':P', 'X'),
 ('PART', 'VERB'),
 ('hey', 'X')]

In [66]:
treebank.tagged_words(tagset='universal')[:10]

[('Pierre', 'NOUN'),
 ('Vinken', 'NOUN'),
 (',', '.'),
 ('61', 'NUM'),
 ('years', 'NOUN'),
 ('old', 'ADJ'),
 (',', '.'),
 ('will', 'VERB'),
 ('join', 'VERB'),
 ('the', 'DET')]

In [67]:
brown_news_tagged = brown.tagged_words(categories='news', tagset='universal')

Quel est le tag le plus fréquent dans ces corpora (e.g. brown)

In [68]:
tag_fd = FreqDist(tag for (word, tag) in brown_news_tagged)

In [70]:
tag_fd

FreqDist({'NOUN': 30654, 'VERB': 14399, 'ADP': 12355, '.': 11928, 'DET': 11389, 'ADJ': 6706, 'ADV': 3349, 'CONJ': 2717, 'PRON': 2535, 'PRT': 2264, ...})

On remarque que NOUN est le genre / tag le plus fréquent dont on peut se servir comme Tag par défaut !

### Exemples de Taggers

Default Tagger permet de tout tagger de la même manière. Si on veut le moins se tromper, choisir <b> NOM </b> comme tag par défaut

In [74]:
from nltk.tag import DefaultTagger

In [75]:
raw = 'I do not like green eggs and ham, I do not like them Sam I am!'
tokens = word_tokenize(raw)

In [76]:
default_tagger = DefaultTagger('NOUN')
default_tagger.tag(tokens)          

[('I', 'NOUN'),
 ('do', 'NOUN'),
 ('not', 'NOUN'),
 ('like', 'NOUN'),
 ('green', 'NOUN'),
 ('eggs', 'NOUN'),
 ('and', 'NOUN'),
 ('ham', 'NOUN'),
 (',', 'NOUN'),
 ('I', 'NOUN'),
 ('do', 'NOUN'),
 ('not', 'NOUN'),
 ('like', 'NOUN'),
 ('them', 'NOUN'),
 ('Sam', 'NOUN'),
 ('I', 'NOUN'),
 ('am', 'NOUN'),
 ('!', 'NOUN')]

Regexp Tagger permet de tagger grâce au Matching avec une expression régulière

In [82]:
from nltk.tag import RegexpTagger

In [83]:
raw = 'Surprisingly, happiness is not always understandable right ?'
tokens = word_tokenize(raw)

In [84]:
regexp_tagger = RegexpTagger([
(r'^-?[0-9]+(.[0-9]+)?$', 'CD'),   # cardinal numbers
(r'(The|the|A|a|An|an)$', 'AT'),   # articles
(r'.*able$', 'JJ'),                # adjectives
(r'.*ness$', 'NN'),                # nouns formed from adjectives
(r'.*ly$', 'RB'),                  # adverbs
(r'.*s$', 'NNS'),                  # plural nouns
(r'.*ing$', 'VBG'),                # gerunds
(r'.*ed$', 'VBD'),                 # past tense verbs
(r'\,|\?', 'PKT'),                 # punctuation
(r'.*', 'NN')                      # nouns (default)
])

In [85]:
regexp_tagger.tag(tokens)

[('Surprisingly', 'RB'),
 (',', 'PKT'),
 ('happiness', 'NN'),
 ('is', 'NNS'),
 ('not', 'NN'),
 ('always', 'NNS'),
 ('understandable', 'JJ'),
 ('right', 'NN'),
 ('?', 'PKT')]

Regardons maintenant un exemple de LookUp Tagger qui tag mot par mot, à savoir Unigram Tagger, en cherchant le tag le plus probable pour ce mot dans le corpus <b> brown </b>

In [92]:
from nltk import ConditionalFreqDist, UnigramTagger

In [93]:
fd = FreqDist(brown.words(categories='news'))
cfd = ConditionalFreqDist(brown.tagged_words(categories='news'))
most_freq_words = list(fd.keys())[:10000]
likely_tags = dict((word, cfd[word].max()) for word in most_freq_words)
baseline_tagger = UnigramTagger(model=likely_tags)

Ici on évalue le tagging en calculant le rapport du mobre de tags bien prédits sur le nombre total de tags sur un ensemble de test à savoir les phrases de <b> brown </b> correspondant à la catégorie <b> romance </b>

In [95]:
baseline_tagger.evaluate(brown.tagged_sents(categories='romance'))

0.7802547770700637

In [96]:
baseline_tagger.tag(word_tokenize('I do not like green eggs and ham, I do not like them Sam I am!'))

[('I', 'PPSS'),
 ('do', 'DO'),
 ('not', '*'),
 ('like', 'CS'),
 ('green', 'NN'),
 ('eggs', 'NNS'),
 ('and', 'CC'),
 ('ham', None),
 (',', ','),
 ('I', 'PPSS'),
 ('do', 'DO'),
 ('not', '*'),
 ('like', 'CS'),
 ('them', 'PPO'),
 ('Sam', 'NP'),
 ('I', 'PPSS'),
 ('am', 'BEM'),
 ('!', '.')]

## Correction orthographique

On peut utiliser TextBlob qui fournit une interface plus simplifiée de nltk tout en se basant sur les mêmes corpora, méthodes, classes... 

In [169]:
%%bash
pip install textblob

Par défaut, seul le dictionnaire anglais est présent; pour les autres langues, par exmeple le Français, penser à télécharger d'autres versions de la lib (e.g. <b> textblob-fr </b> )

In [97]:
from textblob import TextBlob

In [98]:
text = "thiis is teext written with some bad spell"
textblob_text = TextBlob(text)

In [99]:
print(textblob_text.correct())

this is text written with some bad spell


## Stemmification

Regexp Stemmer permet de stemmifier grâce au Matching avec une expression régulière (enlever les parties de mots qui correspondent à la Regex)

In [100]:
from nltk.stem import RegexpStemmer

In [101]:
st = RegexpStemmer('ing$|s$|e$|er$|able$', min=4)

In [102]:
st.stem('porter')

'port'

Lancaster Stemmer permet de stemmifier en se basant sur un algorithme

In [103]:
from nltk.stem.lancaster import LancasterStemmer

In [104]:
st = LancasterStemmer()

In [105]:
st.stem('saying')

'say'

Snowball Stemmer permet de stemmifier en enlevant ses suffixes aux mots. Ces suffixes sont définis en dur dans la définition de la classe <b> nltk.stem.snowball.SnowballStemmer </b> <br>
On peut hériter des Stemmer dans différents langages depuis SnowballStemmer (e.g. FrenchStemmer pour le Français)

In [106]:
from nltk.stem.snowball import FrenchStemmer, SnowballStemmer

In [107]:
fr = FrenchStemmer()

In [108]:
fr.stem('machines')

'machin'

## Lang detection

<b> Langdetect </b> est une lib Python qui permet la reconnaissance de langue d'un paragraphe donné en recherchant chaque mot dans différents dictionnaires de langues. La langue du paragraphe est celle prédominante (le plus de mots dans cette langue)

In [109]:
%%bash
pip install langdetect

In [110]:
import langdetect

In [111]:
text = "This is a text in english les gars"
langdetect.detect(text)

'en'

In [112]:
text = "buongiorno"
langdetect.detect(text)

'tl'

## Exemple simple de pipeline complet

In [113]:
phrase = "Big Data is a domaain that haf attracted many companies and it is introducing techniques such as hadoop and spark"

On tokenise...

In [114]:
phrase_tokens = word_tokenize(phrase)

In [115]:
phrase_tokens

['Big',
 'Data',
 'is',
 'a',
 'domaain',
 'that',
 'haf',
 'attracted',
 'many',
 'companies',
 'and',
 'it',
 'is',
 'introducing',
 'techniques',
 'such',
 'as',
 'hadoop',
 'and',
 'spark']

On détecte la langue de la phrase...

In [116]:
langdetect.detect(phrase)

'en'

On enlève les Stopwords...

In [119]:
clean_phrase_tokens =  [token for token in phrase_tokens if token not in stopwords.words("english")]

In [120]:
clean_phrase_tokens

['Big',
 'Data',
 'domaain',
 'haf',
 'attracted',
 'many',
 'companies',
 'introducing',
 'techniques',
 'hadoop',
 'spark']

On corrige orthographiquement chaque token...

In [121]:
corrected_phrase_tokens =  [list(TextBlob(token).correct().words) for token in clean_phrase_tokens]
corrected_phrase_tokens = [item for sublist in corrected_phrase_tokens for item in sublist]
corrected_phrase_tokens

['Fig',
 'Data',
 'domain',
 'had',
 'attracted',
 'many',
 'companies',
 'introducing',
 'technique',
 'hadoop',
 'spark']

On stemmifie ...

In [122]:
stemmer = SnowballStemmer('english')

In [123]:
stemmed_phrase_tokens = [stemmer.stem(item) for item in corrected_phrase_tokens]

On <em> "dé-tokenise" </em> les tokens pour reconstruire notre phrase transformée...

In [124]:
" ".join(stemmed_phrase_tokens)

'fig data domain had attract mani compani introduc techniqu hadoop spark'

Appliquons maintenant le Pipeline à notre docx du début...

In [125]:
docxtext_tokens = word_tokenize(docxtext.lower())
langdetect.detect(docxtext)

'en'

In [126]:
clean_docxtext_tokens = [
    token for token in docxtext_tokens
    if token not in stopwords.words("english")
    and token not in string.punctuation]

In [127]:
#clean_docxtext_tokens

In [128]:
#pos_tag(clean_docxtext_tokens)

In [129]:
corrected_docx_tokens =  [list(TextBlob(token).correct().words) for token in clean_docxtext_tokens]

In [130]:
corrected_docx_tokens = [item for sublist in corrected_docx_tokens for item in sublist]

Avant la stemmification, voilà la les mots les plus fréquents dans le docx, on remarque que des mots tels que <b> statistical </b> et <b> statistics </b> sont pris séparément alors qu'ils rendent compte d'un même thème à savoir les statistiques...

In [131]:
FreqDist(corrected_docx_tokens)

FreqDist({'data': 290, 'big': 158, 'statistical': 144, 'organizations': 96, 'sources': 48, 'use': 42, 'statistics': 40, 'may': 40, 'information': 38, 'new': 32, ...})

On exécute maintenant la stemmification...

In [132]:
stemmed_docx_tokens = [stemmer.stem(item) for item in corrected_docx_tokens]

In [133]:
stemmed_docx_tokens

['big',
 'big',
 'data',
 'explor',
 'role',
 'big',
 'data',
 'offici',
 'statist',
 'version',
 '0.1',
 'march',
 '2014',
 'draft',
 'review',
 'pleas',
 'note',
 'develop',
 'paper',
 'work',
 'progress',
 'intend',
 'offici',
 'public',
 'reflect',
 'thought',
 'idea',
 'gather',
 'first',
 'virtual',
 'spring',
 'topic',
 'held',
 'march',
 '2014',
 'use',
 'discuss',
 'statist',
 'organ',
 'adapt',
 'potenti',
 'new',
 'futur',
 'realiz',
 'opportun',
 'minim',
 'risk',
 'purpos',
 'paper',
 'encourag',
 'other',
 'join',
 'debat',
 'identifi',
 '‘',
 'big',
 '’',
 'thing',
 'statist',
 'organ',
 'need',
 'tackl',
 'instruct',
 'review',
 'templ',
 'provid',
 'feedback',
 'avail',
 'http',
 'www.once.org/state/platform/display/bigdata/how+big+is+big+data',
 'introduct',
 'big',
 'data',
 'increas',
 'challeng',
 'offici',
 'statist',
 'communiti',
 'need',
 'better',
 'understand',
 'issu',
 'develop',
 'new',
 'method',
 'tool',
 'idea',
 'make',
 'effect',
 'use',
 'big',
 'dat

Après stemmification, les statistiques (racine <b> statist </b>) sont montées plus haut grâce au regroupement ayant été fait sur les mots dérivant de cette racine... ce qui permet de mettre en avant les statistiques comme thème / sujet très caractéristique du document...

In [263]:
FreqDist(stemmed_docx_tokens)

FreqDist({'data': 291, 'statist': 184, 'big': 158, 'organ': 111, 'use': 74, 'sourc': 66, 'need': 47, 'may': 40, 'inform': 40, 'new': 32, ...})